In [1]:
import autograd
import autograd.numpy as np
import autograd.scipy as sp
from autograd.test_util import check_grads

import paragami
import scipy as osp

import example_utils

ImportError: No module named 'example_utils'

In [ ]:
np.random.seed(42)

num_obs = 1000

# True values of parameters
true_sigma = \
    np.eye(3) * np.diag(np.array([1, 2, 3])) + \
    np.random.random((3, 3)) * 0.1
true_sigma = 0.5 * (true_sigma + true_sigma.T)

true_mu = np.array([0, 1, 2])

# Data
x = np.random.multivariate_normal(
    mean=true_mu, cov=true_sigma, size=(num_obs, ))

# Original weights.
original_weights = np.ones(num_obs)

# Original regularizer.
original_lam = 1.0

In [ ]:
def model_logpdf(norm_par, weights, x):
    data_lpdf = example_utils.get_normal_log_prob(
        x, mu=norm_par['mu'], sigma=norm_par['sigma'])
    return np.sum(weights * data_lpdf)

def objective_fun(norm_par, weights, lam, x):
    return \
        -model_logpdf(norm_par, weights, x) + \
        regularizer(norm_par, lam)

norm_par = dict()
norm_par['mu'] = true_mu
norm_par['sigma'] = true_sigma

objective_fun(norm_par, original_weights, original_lam, x)

In [ ]:
norm_param_pattern = paragami.PatternDict()
norm_param_pattern['sigma'] = paragami.PSDSymmetricMatrixPattern(size=3)
norm_param_pattern['mu'] = paragami.NumericArrayPattern(shape=(3, ))

objective_flat = paragami.FlattenFunctionInput(
    lambda par_dict: objective_fun(par_dict, original_weights, original_lam, x),
    patterns=norm_param_pattern,
    free=True)

# It's always a good idea to double check for bugs.

# This checks that the two functions return the same value.
norm_par_flat = norm_param_pattern.flatten(norm_par, free=True)
assert(
    objective_flat(norm_par_flat) == \
    objective_fun(norm_par, original_weights, original_lam, x))

# This verifies that the objective is being differentiated correctly.
check_grads(objective_flat)(norm_par_flat)

In [2]:
objective_wrapper = paragami.OptimizationObjective(objective_flat)
objective_wrapper.set_print_every(5)

init_param = np.ones(norm_param_pattern.flat_length(free=True))
mle_opt = osp.optimize.minimize(
    method='trust-ncg',
    x0=init_param,
    fun=objective_wrapper.f,
    jac=objective_wrapper.grad,
    hessp=objective_wrapper.hessp,
    options={'gtol': 1e-12, 'disp': False})

NameError: name 'objective_flat' is not defined

In [ ]:
print(mle_opt.x)
norm_par_opt = norm_param_pattern.fold(mle_opt.x, free=True)
print(norm_par_opt['sigma'])
print(true_sigma)
print(norm_par_opt['mu'])
print(true_mu)

In [ ]:
# To evaluate the regularization parameter
weight_objective_fun = 

weight_sens = paragami.ParametricSensitivityTaylorExpansion(
    )